In [14]:
import pandas as pd
import seaborn as sns

Reading data

In [15]:
raw_train_data = pd.read_csv('train.csv')
raw_test_data = pd.read_csv('test.csv')

A primitive analysis function, I use to look at data

In [16]:
def my_analysis(dataset):
    df = pd.DataFrame(columns=['Datatype', 'NA values %', 'Unique values', 'mode', 'mode contribution %', 'min value','max value'])
    length = len(dataset.index)
    for col in dataset.columns:
        row_df = pd.DataFrame({
                                'Datatype':dataset[col].dtype,
                                'NA values %':round(dataset[col].isna().sum()*100/length,2),
                                'Unique values':dataset[col].nunique(),
                                'mode':dataset[col].value_counts().index[0],
                                'mode contribution %':round(dataset[col].value_counts()[0]*100/length,2),
                                'min value':'none' if dataset[col].dtype=='object' else dataset[col].min(),
                                'max value':'none' if dataset[col].dtype=='object' else dataset[col].max()
                                },
                                index=[col])
        df = pd.concat([df,row_df])
    return df

In [17]:
my_analysis(raw_train_data)

,Datatype,NA values %,Unique values,mode,mode contribution %,min value,max value
PassengerId,object,0.00,8693,0001_01,0.01,none,none
HomePlanet,object,2.31,3,Earth,52.94,none,none
CryoSleep,object,2.50,2,False,62.57,none,none
Cabin,object,2.29,6560,G/734/S,0.09,none,none
Destination,object,2.09,3,TRAPPIST-1e,68.04,none,none
Age,float64,2.06,80,24.0,2.05,0.0,79.0
VIP,object,2.34,2,False,95.38,none,none
RoomService,float64,2.08,1273,0.0,64.16,0.0,14327.0
FoodCourt,float64,2.11,1507,0.0,62.76,0.0,29813.0
ShoppingMall,float64,2.39,1115,0.0,64.27,0.0,23492.0


In [18]:
my_analysis(raw_test_data)

,Datatype,NA values %,Unique values,mode,mode contribution %,min value,max value
PassengerId,object,0.00,4277,0013_01,0.02,none,none
HomePlanet,object,2.03,3,Earth,52.91,none,none
CryoSleep,object,2.17,2,False,61.73,none,none
Cabin,object,2.34,3265,G/160/P,0.19,none,none
Destination,object,2.15,3,TRAPPIST-1e,69.11,none,none
Age,float64,2.13,79,18.0,1.92,0.0,79.0
VIP,object,2.17,2,False,96.10,none,none
RoomService,float64,1.92,842,0.0,63.74,0.0,11567.0
FoodCourt,float64,2.48,902,0.0,62.89,0.0,25273.0
ShoppingMall,float64,2.29,715,0.0,64.16,0.0,8292.0


As you can see in above analysis, **Cabin** column has so many unique values and replacing the NA values from it can be statistically impossible. Thus we chose to exclude the rows from dataset which have NA values in **Cabin** column.

In [19]:
train_data = raw_train_data[raw_train_data['Cabin'].notna()]

#### Extracting important information from some columns

- **PassengerId** column:\
    As per data description, *'This is an unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.'*\
    We will make two new columns as **Group** and **PeopleId** 
    
    
- **Cabin** column:\
    As per data description, *'The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.'*\
    We will split this column into **Deck**, **Num** and **Side**.


In [20]:
def split_columns(raw_dataset):
    dataset = raw_dataset.copy()

    dataset['Group'] = dataset['PassengerId'].str[:4].astype(float)
    dataset['PeopleId'] = dataset['PassengerId'].str[-2:].astype(float)
    
    dataset['Deck'] = dataset['Cabin'].str[0]
    dataset['Num'] = dataset['Cabin'].str[2:-2].astype(float)
    dataset['Side'] = dataset['Cabin'].str[-1]

    dataset = dataset.drop(['PassengerId','Cabin'], axis=1)
    return dataset


train_data = split_columns(train_data)
test_data = split_columns(raw_test_data)

##### Removing unneccessary columns
Since column **Name** won't really help in training algorithm, we will remove it from our data

In [21]:
remove_columns = ['Name']

train_data = train_data.drop(remove_columns, axis=1)
test_data = test_data.drop(remove_columns, axis=1)

##### Filling Missing Values

From our analysis above we can fill out columns with mode which have less amount of unique values.

In [22]:
mode_columns = ['HomePlanet','Age','CryoSleep','Destination','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

def fillmode(dataset,catcol):
    for col in catcol:
        dataset[col].fillna(dataset[col].mode()[0], inplace=True)

fillmode(train_data,mode_columns)
fillmode(test_data,mode_columns)

##### Categorical columns

Some columns have boolean inputs, we can quickly change into 1's and 0's as following.

In [23]:
boolean_columns = ['CryoSleep','VIP']

def boolean_to_num(dataset,columns):
    for col in columns:
        dataset[col] = dataset[col].astype(int)

boolean_to_num(train_data,boolean_columns)
boolean_to_num(test_data,boolean_columns)

We will perform 'One hot encoding' on categorical columns, basically it will create dummy columns for each category in column

In [24]:
categorical_columns = ['HomePlanet','Destination','Deck','Side']

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
df1 = pd.DataFrame(ohe.fit_transform(train_data[categorical_columns]))
df1.index = train_data.index
train_data = train_data.drop(categorical_columns, axis=1)
train_data = pd.concat([train_data,df1], axis=1)

df2 = pd.DataFrame(ohe.transform(test_data[categorical_columns]))
df2.index = test_data.index
test_data = test_data.drop(categorical_columns, axis=1)
test_data = pd.concat([test_data,df2], axis=1)

Now our data is cleaned and ready to use in any model of our wish

We will save this cleaned data in separate csv file

In [25]:
train_data.to_csv('cleaned_train.csv', index=False)
test_data.to_csv('cleaned_test.csv', index=False)

In [27]:
test_data.isna().sum()

CryoSleep         0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Group             0
PeopleId          0
Num             100
0                 0
1                 0
2                 0
3                 0
4                 0
5                 0
6                 0
7                 0
8                 0
9                 0
10                0
11                0
12                0
13                0
14                0
15                0
dtype: int64